In [ ]:
# Packages needed:
#   StatsBase
#   PyPlot
include("example_common.jl");

In [ ]:
# Column-stochastic random walk matrix
P1 = [[0, 0.5, 0.5] [0.5, 0, 0.5] [0.5, 0.5, 0]];

# Simulate the first-order Markov chain, starting at node 1
starting_node = 1
(counts1, hist1) = sim_FOMC(P1, starting_node)
counts1 / sum(counts1), hist1[1:50]

In [ ]:
# Nonbacktracking random walk transition probability tensor
P2 = zeros(Float64, 3, 3, 3);
P2[:, 1, 2] = [0, 0, 1.0]; P2[:, 2, 1] = [0, 0, 1.0]
P2[:, 1, 3] = [0, 1.0, 0]; P2[:, 3, 1] = [0, 1.0, 0]
P2[:, 2, 3] = [1.0, 0, 0]; P2[:, 3, 2] = [1.0, 0, 0]

# Simulate the second-order Markov chain starting at pair (1, 2)
starting_pair = (1, 2)
(counts2, hist2) = sim_SOMC(P2, starting_pair)
counts2 / sum(counts2), hist2[1:50]

In [ ]:
# Spacey random walks!
n = size(P2, 1)

# Set undefined transitions to be uniform
undef_a = ones(Float64, n) / n
(counts3a, hist3a) = sim_SOSRW(P2, starting_node, undef_a)
dist3a = counts3a / sum(counts3a)

In [ ]:
# Set undefined transitions to go back to 1
undef_b = zeros(Float64, n); 
undef_b[1] = 1.0
(counts3b, hist3b) = sim_SOSRW(P2, starting_node, undef_b)
dist3b = counts3b / sum(counts3b)

In [ ]:
# Use the super spacey random walk
(counts3s, hist3s) = sim_SOSRW(P2, starting_node, Float64[], true)
counts3s / sum(counts3s)

In [ ]:
# Let's check that we are actually getting out eigenvectors!

# First, fill in the zero columns.
P2a = copy(P2)
P2b = copy(P2)
n = size(P2, 1)
for j = 1:n, k = 1:n
    col = vec(P2[:, j, k])
    if maximum(col) == 0.0
        P2a[:, j, k] = undef_a
        P2b[:, j, k] = undef_b
    end
end

# Apply the stationary distributions to the tensors
ya = apply(P2a, dist3a)
yb = apply(P2b, dist3b)

@show minimum(ya ./ dist3a), maximum(ya ./ dist3a)
@show minimum(yb ./ dist3b), maximum(yb ./ dist3b)
;

In [ ]:
# Done!